# **Notes** 
### Purpose: Calculate the total trend in TXx and TXm for ERA5

# **Preparation**

In [ ]:
import numpy as np # for storing vector and matrix data
import matplotlib.pyplot as plt # to plot figures
import netCDF4 as nc #to read netCDF files
import cartopy.crs as ccrs # to plot maps
# (ergens in test ook: import cartopy as cart)
import cartopy.feature as cf
# from matplotlib import ticker
import scipy.io
from scipy.stats import pearsonr # voor persistence
import scipy.stats as stats
# from cartopy.util import add_cyclic_point
import os
from datetime import datetime, timedelta
import xarray as xr
import pickle

# **Functions**

In [ ]:
def load_data(path,*variables_to_add):
  """
Provide the path to a file and the variables you want to extract
  """
  data = nc.Dataset(path, mode='r')
  variable_list = []
  for variable in variables_to_add:
    var =data.variables[variable][:]
    variable_list.append(var)
  return variable_list

In [ ]:
def plot_variablet(lat, lon, variable,folder,name):
    plt.figure(figsize = (10,10))
    ax = plt.axes(projection = ccrs.PlateCarree())
    plot = plt.contourf(lon, lat, variable, cmap = "RdBu_r", transform = ccrs.PlateCarree(), levels = 15) #levels=np.linspace(-8.2e7, 1e7, 10), extend='both
    ax.coastlines()
    ax.add_feature(cf.BORDERS)
    plt.colorbar(plot, ax=ax, orientation = "horizontal", label = "degree celcius/GWD", pad = 0.05)
    #plt.savefig(f"{folder}/{name}.png",dpi=300)
    plt.show()
    plt.close()

In [ ]:
def extract_area(S, N, W, E, lat, lon, variable,event = False):
    """
    This function slices the data given the S, N, W, E bounds. Use event = True if there are only two dimensions (since then there is no time dimension), this means after using this
    function you need to use event[0] to get the data
    """
    # Change longitude data to go from -180 to 180
    for i in range(len(lon)):
        if lon[i] > 180:
          lon[i] = lon[i] - 360
        else:
          lon[i] = lon[i]

    # Calculate the index of the bounds
    sIndex = np.argmin(np.abs(lat - S))
    nIndex = np.argmin(np.abs(lat - N))
    wIndex = np.argmin(np.abs(lon - W))
    eIndex = np.argmin(np.abs(lon - E))

    if event:
        variable = np.expand_dims(variable, axis = 0)

    if wIndex > eIndex: # If the west index is higher than the east index, think of the right side of the world map as left boundary and vice versa
        latSlice = lat[sIndex: nIndex + 1]
        lonSlice = np.concatenate((lon[wIndex:], lon[:eIndex + 1]))
        variableSlice = np.concatenate((variable[:, sIndex: nIndex + 1, wIndex:], variable[:, sIndex: nIndex + 1, :eIndex + 1]), axis = 2)

    else:
        latSlice = lat[sIndex: nIndex + 1]
        lonSlice = lon[wIndex: eIndex + 1]
        variableSlice = variable[:, sIndex: nIndex + 1, wIndex: eIndex + 1]

    return latSlice, lonSlice, variableSlice

In [ ]:
def extract_season_year(variable,yearlength,start_day,end_day, start_year = False, end_year = False):
  """
  Start day and end day should be the actuall day, so if you want the second day, third and fourth day, do 2,4 (151,240 would be JJA?) jaren ook: 1,5 is 1 tot en met 5
  nadenken dat als gaat checken met al gesneden data dat yearlenght 90 is als op 3 maanden gesneden
  """
  start_index = start_day-1
  end_index = end_day-1
  if start_year == False and end_year == False:
    years = variable.shape[0]//yearlength
    for year in range(years):
      if year == 0:
        selected_data = variable[(year*yearlength)+start_index:(end_index+1),:,:] # +1 omdat tot is ipv tot en met voor de laatste
      elif year != 0:
        add_data = variable[(year*yearlength)+start_index:(year*yearlength)+(end_index+1),:,:] # stel is 10, na 1 jaar dan 370 is TOT 370 dus index 369 en dan is dag 10
        selected_data = np.concatenate((selected_data, add_data), axis = 0)
    return selected_data
  else:
    years = (end_year-start_year) + 1
    for year in range(years):
      year_multiplier = (year + start_year) - 1
      if year == 0:
        selected_data = variable[(year_multiplier*yearlength)+start_index:(year_multiplier*yearlength)+(end_index+1),:,:] # +1 omdat tot is ipv tot en met voor de laatste
      elif year != 0:
        add_data = variable[(year_multiplier*yearlength)+start_index:(year_multiplier*yearlength)+(end_index+1),:,:] # stel is 10, na 1 jaar dan 370 is TOT 370 dus index 369 en dan is dag 10
        selected_data = np.concatenate((selected_data, add_data), axis = 0)
    return selected_data

In [ ]:
def global_warming_degree_function(temp_data,adjusted_yearlength, start_year, end_year):
    """
    Hardcodes that its a 5 year centred running average, moet temp_data van seizoen of hele jaar?
    """
    amount_of_years = (end_year - start_year) + 1
    list_for_finalized_averages = []
    list_for_GWD = []
    reshaped_data = temp_data.reshape(amount_of_years,adjusted_yearlength,temp_data.shape[1],temp_data.shape[2])
    for i in range(amount_of_years):
        if i == 0:
            filtered_data = reshaped_data[0:3,:,:,:]
        elif i == 1:
            filtered_data = reshaped_data[0:4,:,:,:]
        elif i == (amount_of_years-1): #laatste
            filtered_data = reshaped_data[(i-2):,:,:,:]
        elif i == (amount_of_years-2): 
            filtered_data = reshaped_data[(i-2):,:,:,:]
        else:
            filtered_data = reshaped_data[(i-2):(i+3),:,:,:]
        mean_to_add = np.mean(filtered_data) # zou alle axis moeten meanen
        list_for_finalized_averages.append(mean_to_add)

    for average in list_for_finalized_averages:
        GWD_value = average - list_for_finalized_averages[-1]
        list_for_GWD.append(GWD_value)
    
    return list_for_GWD

In [ ]:
def temperature_trends__TXx_TXm(max_temp_data,adjusted_yearlength, start_year, end_year):
    """
    Needs to use the maximum daily temperature data (TASMAX),note that instead of looping through the years, you could also just do np.mean(...,axis = 1)
    because the o.g. reshaped data is still (years,months,lat,lon) (or lon/lat), so than you would get (years,lat,lon) 
    """
    amount_of_years = (end_year - start_year) + 1
    reshaped_data = max_temp_data.reshape(amount_of_years,adjusted_yearlength,max_temp_data.shape[1],max_temp_data.shape[2])
    list_for_Txx_values = []
    list_for_Txm_values = []

    for i in range(amount_of_years):
        filtered_data = reshaped_data[i,:,:,:]
        max_per_season = np.max(filtered_data,axis = 0)
        list_for_Txx_values.append(max_per_season)
        mean_per_season = np.mean(filtered_data, axis = 0)
        list_for_Txm_values.append(mean_per_season)

    array_for_TXx = np.array(list_for_Txx_values)
    array_for_TXm = np.array(list_for_Txm_values)
    
    return array_for_TXx, array_for_TXm

In [ ]:
def regression_temperature_trend(GWD_list,temperature_trend_array):

    # Create empty arrays for the regression outputs
    shape_1 = temperature_trend_array.shape[1]
    shape_2 = temperature_trend_array.shape[2]

    slope_array = np.zeros((shape_1,shape_2))
    intercept_array = np.zeros((shape_1,shape_2))
    rvalue_array = np.zeros((shape_1,shape_2))
    pvalue_array = np.zeros((shape_1,shape_2))
    stderr_array = np.zeros((shape_1,shape_2))

    for i in range(shape_1):
        for j in range(shape_2):
            values_at_specific_coordinates = temperature_trend_array[:, i, j] # Find the values for all years in 1 grid cell
            slope, intercept, rvalue, pvalue, stderr = stats.linregress(GWD_list, values_at_specific_coordinates)
            
            # Store the regression outputs in the empty arrays
            slope_array[i, j] = slope
            intercept_array[i, j] = intercept
            rvalue_array[i, j] = rvalue
            pvalue_array[i, j] = pvalue
            stderr_array[i, j] = stderr

    return slope_array,intercept_array,rvalue_array,pvalue_array,stderr_array

In [ ]:
def lists_for_era5_dates(final_year,final_month,final_day):
    """
    Creates lists with all months and years in the complete ERA5 data, taking into account leap days (schrikkeldagen)
    """
    start_date_all_era5_data = datetime(1950, 1, 1) # Is included
    end_date_all_era5_data = datetime(final_year,final_month,final_day) # Is included
    delta_time = timedelta(days=1)

    date_list_basic = []
    current_date = start_date_all_era5_data
    while current_date <= end_date_all_era5_data:
        date_list_basic.append(current_date)
        current_date += delta_time
    #date_strings = [date.strftime('%Y-%m-%d') for date in date_list_basic]
    month_list = [date.month for date in date_list_basic]
    year_list = [date.year for date in date_list_basic]

    return month_list, year_list


In [ ]:
def extract_years_and_months_era5(era5_data,desired_start_month,desired_end_month,desired_start_year,desired_end_year,list_with_all_months,list_with_all_years):
    """
    Slice era5 data based on the months and years, months and years that are used as input variable will be included as well
    """
    if era5_data.shape[0] != len(list_with_all_months):
        print ("Error: Amount of days in the data and list with all dates are not the same")
    
    list_for_filtered_era5_data = []
    for i in range(era5_data.shape[0]):
        month_at_index = list_with_all_months[i]
        year_at_index = list_with_all_years[i]
        if month_at_index >= desired_start_month and month_at_index <= desired_end_month and year_at_index >= desired_start_year and year_at_index <= desired_end_year:
            data_to_select = era5_data[i,:,:]
            list_for_filtered_era5_data.append(data_to_select)
            
    array_selected_era5_data = np.array(list_for_filtered_era5_data)

    return array_selected_era5_data

In [ ]:
def global_warming_degree_function_era5(temp_data,list_of_years, start_year, end_year):
    """
    Hardcodes that its a 5 year centred running average, moet temp_data van seizoen of hele jaar?
    laatste if niet nodig want zegt dat maar 65 jaar doet (tot 2014) maar era5 data en year list lopen tot 2024 dus de year list loopt gwn naar 2015
    """
    amount_of_years = (end_year - start_year) + 1
    list_for_finalized_averages = []
    list_for_GWD = []

    list_for_reshaped_data = []
    day = 0
    for i in range(amount_of_years):
        list_for_data_per_year = []
        condition_variable = True
        while condition_variable == True:
            data_to_add = temp_data[day,:,:]
            list_for_data_per_year.append(data_to_add)
            day = day + 1
            if list_of_years[day] != list_of_years[day-1]:
                list_for_reshaped_data.append(list_for_data_per_year)
                condition_variable = False
            #if day == (temp_data.shape[0]): #niet -1 omdat hierboven al day + 1 hebt gedaan
                #condition_variable = False

    for i in range(amount_of_years):
        if i == 0:
            indexes = [0,1,2]
        elif i == 1:
            indexes = [0,1,2,3]
        elif i == (amount_of_years-1): #laatste
            indexes = [i-2,i-1,i]
        elif i == (amount_of_years-2): #(ook i nog minder hoog dus drm zelfde als vorige)
            indexes = [i-2,i-1,i,i+1]
        else:
            indexes = [i-2,i-1,i,i+1,i+2]
        
        list_for_means_per_year = []
        for index in indexes:
            list_to_analyse = list_for_reshaped_data[index]
            array_to_analyse = np.array(list_to_analyse)
            mean_for_year = np.mean(array_to_analyse)
            list_for_means_per_year.append(mean_for_year)
        combined_mean = (sum(list_for_means_per_year))/(len(list_for_means_per_year))
        list_for_finalized_averages.append(combined_mean)

    for average in list_for_finalized_averages:
        GWD_value = average - list_for_finalized_averages[-1]
        list_for_GWD.append(GWD_value)
    
    return list_for_GWD

In [ ]:
def mean_weighted(data):
    """
    Parameters
    ----------
    data : data
         data.
    """
    ## Calculate global-mean surface temperature (GMST)
    cos_lat_2d = np.cos(np.deg2rad(data['lat'])) * xr.ones_like(data['lon']) # effective area weights
    mean_ = ((data * cos_lat_2d).sum(dim=['lat','lon']) /
                 cos_lat_2d.sum(dim=['lat','lon']))
    return mean_

In [ ]:
def global_warming_degree_function_era5_area_weighted(temp_data_path,list_of_years, start_year, end_year, variable = "t2m"):
    """
    Hardcodes that its a 5 year centred running average, moet temp_data van het hele jaar
    laatste if niet nodig want zegt dat maar 65 jaar doet (tot 2014) maar era5 data en year list lopen tot 2024 dus de year list loopt gwn naar 2015
    doet weighted mean van de gegeven variable, hardcodes dat lat en lon heet (dus als variables in data lattitude en longitude hebt = aanpasses)
    door for i in range(amount_of_years) hardcodes ook dat in 1950 begint!
    """
    #Open the NetCDF file with xarray
    dataset = xr.open_dataset(temp_data_path)

    #Calculate the area weighted mean
    weighted_mean = mean_weighted(dataset[variable])

    #Transform the xarray DataArray into a list
    daily_mean_list = weighted_mean.values.tolist()


    amount_of_years = (end_year - start_year) + 1
    list_for_finalized_averages = []
    list_for_GWD = []

    list_for_reshaped_data = []
    day = 0
    for i in range(amount_of_years):
        list_for_data_per_year = []
        condition_variable = True
        while condition_variable == True:
            data_to_add = daily_mean_list[day]
            list_for_data_per_year.append(data_to_add)
            day = day + 1
            if list_of_years[day] != list_of_years[day-1]: #deze vergelijken met vorige omdat al day+1 hebt gedaan
                list_for_reshaped_data.append(list_for_data_per_year)
                condition_variable = False
            #if day == (temp_data.shape[0]): #niet -1 omdat hierboven al day + 1 hebt gedaan
                #condition_variable = False

    for i in range(amount_of_years):
        if i == 0:
            indexes = [0,1,2]
        elif i == 1:
            indexes = [0,1,2,3]
        elif i == (amount_of_years-1): #laatste
            indexes = [i-2,i-1,i]
        elif i == (amount_of_years-2): #(ook i nog minder hoog dus drm zelfde als vorige)
            indexes = [i-2,i-1,i,i+1]
        else:
            indexes = [i-2,i-1,i,i+1,i+2]
        
        list_for_means_per_year = []
        for index in indexes:
            list_to_analyse = list_for_reshaped_data[index]
            array_to_analyse = np.array(list_to_analyse)
            mean_for_year = np.mean(array_to_analyse)
            list_for_means_per_year.append(mean_for_year)
        combined_mean = (sum(list_for_means_per_year))/(len(list_for_means_per_year))
        list_for_finalized_averages.append(combined_mean)

    for average in list_for_finalized_averages:
        GWD_value = average - list_for_finalized_averages[-1]
        list_for_GWD.append(GWD_value)
    
    return list_for_GWD

In [ ]:
def mean_weighted_masked(data,mask):
    """
    Parameters
    ----------
    data : data
         data.
    """
    ## Calculate global-mean surface temperature (GMST)
    mask_af = np.squeeze(mask)
    cos_lat_2d = np.cos(np.deg2rad(data['lat'])) * xr.ones_like(data['lon']) # effective area weights
    if cos_lat_2d.shape != mask_af.shape:
        print (f"WARNING: shapes of mask ({mask_af}) and cos_lat2d  ({cos_lat_2d.shape}) do no match (in mean_weigted2 (zelf))")
    cos_lat_2d_masked = cos_lat_2d*mask_af
    mean_ = ((data * cos_lat_2d).sum(dim=['lat','lon']) /
                 cos_lat_2d_masked.sum(dim=['lat','lon']))
    return mean_

In [ ]:
def area_averaged_trend(data,lat,lon,landmask_path,S,N,W,E):
    """
    Calculates the area weighted mean land area trend of a region using a land mask
    """

    #Load the mask and extract the selected region for the mask and actual data
    latm,lonm,mask_data = load_data(landmask_path,"lat","lon","tx")
    latm_box,lonm_box,mask_box = extract_area(S,N,W,E,latm,lonm,mask_data,event = False)
    latd_box,lond_box,data_box = extract_area(S,N,W,E,lat,lon,data,event = True)

    #Prepare mask
    min_data = np.min(mask_box)
    if min_data == -9999.0:
        mask_box = np.where(mask_box == -9999.0, np.nan, mask_box)
    mask = mask_box/mask_box
    if mask.shape != data_box.shape:
        print ("WARNING: shape of mask and data do not match (zelf)")
    
    #Perform analyses and plot to check
    masked_data = data_box*mask
    plot_variablet(latd_box,lond_box,masked_data[0],"test","test")

    #Turn into xarray and take area weighted average
    time_indices = np.arange(masked_data.shape[0])  # Because you used event = True in extract area both now have three dimensions
    data_xr_array = xr.DataArray(masked_data, dims=['time', 'lat', 'lon'], coords={'time': time_indices, 'lat': latd_box, 'lon': lond_box})
    mean_trend = mean_weighted_masked(data_xr_array,mask)
    mean_list = mean_trend.values.tolist()
    print (f"weighted mean = {mean_list}, normal mean = {np.mean(masked_data)}")

    return mean_list
    

# **Set-up**

In [ ]:
#Select: "regridded" or "original", Als original data doet hebt kans dat kleiner gebied moet doen want doet EU data en dat data geupdate is dus dat lijst langer moet zijn met dates
data_to_use = "regridded"

In [ ]:
#The latest date in the original ERA5 dataset, (for the regridded data its 2024, 2, 29 which is selected automatically if data_to_use = "regridded" 
final_year_og = 2024
final_month_og = 3
final_day_og = 31

In [ ]:
#Select the months and years that will be analysed
desired_start_monthV = 6 # 6 voor JJA en 3 voor MAM
desired_end_monthV = 8 # 8 voor JJA en 5 voor MAM
desired_start_yearV = 1950 # 1950 for all data
desired_end_yearV = 2023 # 2014 for model comparison, 2023 is the last complete year

In [ ]:
#Select the area to be analysed and the area over which the average trend is calculated
bboxT = [30,70,-30,30] #Pick borders like: [S,N,W,E] (T from temperature) for serious_run1 = [30,70,-30,30]
bboxA = [45,55,-5,15] #Pick borders like: [S,N,W,E] (A from average) for Vautard = [45,55,-5,15]

In [ ]:
# Set the adjusted yearlenght
adjusted_yearlengthV = 92 # 92 voor JJA en MAM

In [ ]:
# Where and how to save (zelf mapje al maken! (in prompt mkdir ...))
save_outputV = True
plot_outputV = True
save_pathV = '/usr/people/noest/stage_folders/outputs/net/serious_run2/total_trend/era5'
season_nameV = "JJA_new" #JJA or MAM
save_statisticsV = True
save_path_statisticsV = '/usr/people/noest/stage_folders/outputs/net/serious_run2/total_trend/era5'

# **Uitvoeren**

In [ ]:
if data_to_use == "regridded":
    data_path_tmax = "/net/pc200246/nobackup/users/noest/ERA5_regridded/era5_tmax_daily_regridded.nc"
    data_path_t2m = "/net/pc200246/nobackup/users/noest/ERA5_regridded/era5_t2m_daily_regridded.nc"
    land_mask_path = "/net/pc200246/nobackup/users/noest/landmask/landmask_day_regridded.nc"
    final_yearV = 2024
    final_monthV = 2
    final_dayV = 29
elif data_to_use == "original":
    data_path_tmax = "/net/pc230042/nobackup/users/sager/nobackup_2_old/ERA5-CX-READY/era5_tmax_daily_eu.nc"
    #data_path_t2m = "/net/pc230042/nobackup/users/sager/nobackup_2_old/ERA5-CX-READY/era5_t2m_daily.nc"
    data_path_t2m = "/net/pc200246/nobackup/users/noest/ERA5_regridded/era5_t2m_daily_regridded.nc"
    land_mask_path = "/net/pc200246/nobackup/users/noest/landmask/landmask_day_tmax_highres.nc"
    final_yearV = final_year_og
    final_monthV = final_month_og
    final_dayV = final_day_og

In [ ]:
#Determine the boundaries to use for the temperature trend
if data_to_use == "regridded":
    S1 = bboxT[0] # for serious_run1 = 30 en 70 als data_to_use = "original"
    N1 = bboxT[1] # for serious_run1 = 70 en 30 als data_to_use = "original"
    W1 = bboxT[2] # for serious_run1 = -30
    E1 = bboxT[3] # for serious_run1 = 30
elif data_to_use == "original":
    S1 = bboxT[1] # for serious_run1 = 30 en 70 als data_to_use = "original"
    N1 = bboxT[0] # for serious_run1 = 70 en 30 als data_to_use = "original"
    W1 = bboxT[2] # for serious_run1 = -30
    E1 = bboxT[3] # for serious_run1 = 30

In [ ]:
#Determine the boundaries to use for the average trend
if data_to_use == "regridded":
    S_A = bboxA[0] # for serious_run1 = 30 en 70 als data_to_use = "original"
    N_A = bboxA[1] # for serious_run1 = 70 en 30 als data_to_use = "original"
    W_A = bboxA[2] # for serious_run1 = -30
    E_A = bboxA[3] # for serious_run1 = 30
elif data_to_use == "original":
    S_A = bboxA[1] # for serious_run1 = 30 en 70 als data_to_use = "original"
    N_A = bboxA[0] # for serious_run1 = 70 en 30 als data_to_use = "original"
    W_A = bboxA[2] # for serious_run1 = -30
    E_A = bboxA[3] # for serious_run1 = 30

In [ ]:
#t2m_lat,t2m_lon,t2m_global = load_data(data_path_t2m,"lat","lon","t2m")

In [ ]:
tmax_lat,tmax_lon,tmax = load_data(data_path_tmax,"lat","lon","tmax")

In [ ]:
list_with_months, list_with_years = lists_for_era5_dates(final_yearV,final_monthV,final_dayV)

In [ ]:
tmax_JJA = extract_years_and_months_era5(tmax,desired_start_monthV,desired_end_monthV,desired_start_yearV,desired_end_yearV,list_with_months,list_with_years)

In [ ]:
lat_slice,lon_slice,tmax_extracted = extract_area(S1, N1, W1, E1, tmax_lat,tmax_lon, tmax_JJA,event = False)

In [ ]:
Txx_era5, Txm_era5 = temperature_trends__TXx_TXm(tmax_extracted,adjusted_yearlengthV, desired_start_yearV, desired_end_yearV)

In [ ]:
gwd_era5 = global_warming_degree_function_era5_area_weighted(data_path_t2m,list_with_years,desired_start_yearV,desired_end_yearV)

In [ ]:
slope_array_era5_Txx,intercept_array_era5_Txx,rvalue_array_era5_Txx,pvalue_array_era5_Txx,stderr_array_era5_Txx = regression_temperature_trend(gwd_era5,Txx_era5)

In [ ]:
slope_array_era5_Txm,intercept_array_era5_Txm,rvalue_array_era5_Txm,pvalue_array_era5_Txm,stderr_array_era5_Txm = regression_temperature_trend(gwd_era5,Txm_era5)

In [ ]:
if save_outputV == True:
    np.save(f"{save_pathV}/{season_nameV}_era5_TXx_slope_{data_to_use}_data_until_{desired_end_yearV}.npy",slope_array_era5_Txx) 
    np.save(f"{save_pathV}/{season_nameV}_era5_TXm_slope_{data_to_use}_data_until_{desired_end_yearV}.npy",slope_array_era5_Txm)
    lat_to_save = np.array(lat_slice)
    lon_to_save = np.array(lon_slice)
    np.save(f"{save_pathV}/{season_nameV}_era5_lat_extracted_{data_to_use}_data_until_{desired_end_yearV}.npy",lat_to_save) 
    np.save(f"{save_pathV}/{season_nameV}_era5_lon_extracted_{data_to_use}_data_until_{desired_end_yearV}.npy",lon_to_save)

In [ ]:
if save_statisticsV == True:
    np.save(f"{save_path_statisticsV}/{season_nameV}_era5_TXx_stderr_{data_to_use}_data_until_{desired_end_yearV}.npy",stderr_array_era5_Txx) 
    np.save(f"{save_path_statisticsV}/{season_nameV}_era5_TXx_pvalue_{data_to_use}_data_until_{desired_end_yearV}.npy",pvalue_array_era5_Txx) 
    np.save(f"{save_path_statisticsV}/{season_nameV}_era5_TXm_stderr_{data_to_use}_data_until_{desired_end_yearV}.npy",stderr_array_era5_Txm) 
    np.save(f"{save_path_statisticsV}/{season_nameV}_era5_TXm_pvalue_{data_to_use}_data_until_{desired_end_yearV}.npy",pvalue_array_era5_Txm) 

In [ ]:
if plot_outputV == True:
    plot_variablet(lat_slice,lon_slice,slope_array_era5_Txx,"tset","test")
    plot_variablet(lat_slice,lon_slice,slope_array_era5_Txm,"tset","test")

In [ ]:
mean_Txx = area_averaged_trend(slope_array_era5_Txx,lat_slice,lon_slice,land_mask_path,S_A,N_A,W_A,E_A)
mean_Txm = area_averaged_trend(slope_array_era5_Txm,lat_slice,lon_slice,land_mask_path,S_A,N_A,W_A,E_A)

In [ ]:
if save_outputV == True:
    filenametxx = f"{save_pathV}/{season_nameV}_era5_meanTxxlist_{data_to_use}_data_until_{desired_end_yearV}.pkl"
    with open(filenametxx, 'wb') as f:
        pickle.dump(mean_Txx, f)
    filenametxm = f"{save_pathV}/{season_nameV}_era5_meanTxmlist_{data_to_use}_data_until_{desired_end_yearV}.pkl"
    with open(filenametxm, 'wb') as f:
        pickle.dump(mean_Txm, f)

In [ ]:
# def load_lists_from_file(measurement_name, stack_name):
#     filename = f"/content/drive/MyDrive/Colab Notebooks/Research Project/plume_output/{measurement_name}_{stack_name}.pkl"
#     with open(filename, 'rb') as f:
#         loaded_lists = pickle.load(f)
#     return loaded_lists

In [ ]:
#np.save(f"/usr/people/noest/stage_folders/outputs/dynamic_test/TOTAL_Txx_weighted2022.npy",slope_array_era5_Txx) 
#np.save(f"/usr/people/noest/stage_folders/outputs/dynamic_test/TOTAL_Txm_weighted2022.npy",slope_array_era5_Txm) 